In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

from numpy.random import default_rng
# stabilité du notebook d'une exécution à l'autre

random=default_rng(42)
# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID) # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)



In [10]:
import os
import tarfile
import urllib.request
from pathlib import Path
import pathlib
print(pathlib.Path().resolve(), " put all the cloned files in the same folder")

# General data
general_data = pd.read_csv('general_data.csv',dtype={'EmployeeID': object})
# survey data 
manager_survey_data = pd.read_csv('manager_survey_data.csv',dtype={'EmployeeID': object})
employee_survey_data = pd.read_csv('employee_survey_data.csv',dtype={'EmployeeID': object})
# office entry/exit time
in_time = pd.read_csv('in_time.csv',dtype={'EmployeeID': object})
out_time = pd.read_csv('out_time.csv',dtype={'EmployeeID': object})

print(general_data.describe())


C:\Users\lyder\Desktop\CESI\FISE A4 INFO\BLOC AI\Projet\blocAI  put all the cloned files in the same folder
               Age  DistanceFromHome    Education  EmployeeCount     JobLevel  \
count  4410.000000       4410.000000  4410.000000         4410.0  4410.000000   
mean     36.923810          9.192517     2.912925            1.0     2.063946   
std       9.133301          8.105026     1.023933            0.0     1.106689   
min      18.000000          1.000000     1.000000            1.0     1.000000   
25%      30.000000          2.000000     2.000000            1.0     1.000000   
50%      36.000000          7.000000     3.000000            1.0     2.000000   
75%      43.000000         14.000000     4.000000            1.0     3.000000   
max      60.000000         29.000000     5.000000            1.0     5.000000   

       MonthlyIncome  NumCompaniesWorked  PercentSalaryHike  StandardHours  \
count    4410.000000         4391.000000        4410.000000         4410.0   
mean  